In [3]:
import sys
sys.path.append('../../30_data_tools/')

In [4]:
import pandas as pd
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from pathlib import Path
import numpy as np
import sqlite3
import json

In [5]:
from helper import load_dotenv

# Qualität der generierten Muster bewerten

Dieses Notebook dient zur Erkundung, ob bewertet werden kann, ob eine erzeugte Kachel ein sinnvolles Muster zur Grafik hinzufügt oder nicht.
Dafür wird die erzeugte Kachel mit der Orignalkachel mit Hilfe der structured similarity (SSIM) verglichen.

In [7]:
config = load_dotenv()

In [ ]:
con = sqlite3.connect(config['DB_PATH'])

In [ ]:
images = pd.read_sql(
    '''
        SELECT cf.*, mf.mask_filename FROM (
        	SELECT * FROM related_file
        	WHERE variant_name = 'halftone300dpi' AND "type" = '4c'
        ) cf
        LEFT JOIN (
        	SELECT job, pdf_filename, filename AS mask_filename, 1 AS has_mask FROM related_file 
        	WHERE variant_name = 'halftone300dpi' AND "type" = 'masks'
        ) mf ON cf.job=mf.job AND cf.pdf_filename=mf.pdf_filename 
        WHERE mf.has_mask IS NOT NULL
    ''',
    con
)

images.loc[
    :,
    'image'
] = images.filename.str.replace('.jpg','', regex=True)

images = images.loc[
    images.job.str.match('\d{6}')
]

In [9]:
data = pd.concat([pd.read_pickle(pkl_file) for pkl_file in config['GENERIC_INFORMATION_DATA_DIR'].glob('./*.pkl')])

In [14]:
data.loc[
    data.basic_name.str.contains('508890.HBZP_1_M_269.p1')
]

,image,img_path,method,ssim,bbox,use_blow_up_centered,use_contract_centered,use_pillow_disortion,use_roll,use_rotation,...,basic_name,scale,trapezoidal_distortion_strength_top,trapezoidal_distortion_strength_bottom,trapezoidal_distortion_direction,trapezoidal_distortion_strength,stretch_x,stretch_y,rotation_degree,centered_c
55,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999980,"[930, 2836, 1286, 980]",False,True,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,10.0
56,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999691,"[736, 1278, 1818, 2086]",False,False,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,NaN,NaN,NaN,NaN,1.241269,1.0,45.0,NaN
57,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999463,"[2034, 2824, 626, 986]",False,False,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,0.066172,0.018987,right,NaN,NaN,NaN,45.0,NaN
58,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999860,"[438, 2820, 2222, 992]",False,True,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,10.0
59,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999802,"[440, 978, 2224, 2116]",False,True,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,10.0
60,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.814300,"[716, 2460, 716, 896]",False,False,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,1.5,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN
61,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.805999,"[438, 2992, 638, 814]",False,False,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN
62,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999999,"[5310, 0, 252, 252]",False,False,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,1.5,NaN,NaN,right,0.059325,NaN,NaN,45.0,NaN
63,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.799824,"[564, 1040, 290, 454]",False,False,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN
64,HBZP_1_M_269.p1.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999999,"[0, 6856, 238, 242]",True,False,False,False,True,...,508890.HBZP_1_M_269.p1.$PLACEHOLDER$.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,10.0


In [8]:
data = pd.concat([pd.read_pickle(pkl_file) for pkl_file in config['GENERIC_INFORMATION_DATA_DIR'].glob('./*.pkl')])

data = pd.merge(
    data,
    images.loc[
        :,
        ['image','filename', 'variant_name','job']
    ],
    how="left",
    on="image"
)

NameError: name 'images' is not defined

In [ ]:
def measure_similarity( row ):
    orig_img_path = config['DATA_DIR'] / row.job / row.variant_name / row.filename
    generated_img_path = config['GENERIC_GENERATED_DATA_DIR'] / f"{row.basic_name.replace('$PLACEHOLDER$',row.method)}.jpg"
    masks_path = config['GENERIC_GENERATED_DATA_DIR'] / f"{row.basic_name.replace('$PLACEHOLDER$','masks')}.json"

    if orig_img_path.exists() == False or generated_img_path.exists() == False or masks_path.exists() == False:
        raise "file does not exist"
    
    with masks_path.open() as masks_file:
        masks = json.load(masks_file)

    orig_img = np.array(Image.open(orig_img_path))
    generated_img = np.array(Image.open(generated_img_path))
    out = []
    
    for m in masks:
        ssim_value = ssim(
            orig_img[
                m[1]:m[1]+m[3],
                m[0]:m[0]+m[2],
                3
            ],
            generated_img[
                m[1]:m[1]+m[3],
                m[0]:m[0]+m[2],
                3
            ]
        )

        out.append((
            ssim_value,
            generated_img[
                m[1]:m[1]+m[3],
                m[0]:m[0]+m[2],
                3
            ]
        ))

    return out

In [ ]:
filtered_data = data.loc[
    :,
    ['image','method','basic_name','filename','variant_name','job']
].drop_duplicates()

In [ ]:
tiles = []

for idx in filtered_data.sample(n=25).index:
    row = filtered_data.loc[idx]

    row_results = measure_similarity(row)
    tiles += [
        (rr[0],rr[1], row.basic_name)
        for rr in row_results
    ]

In [ ]:
def show_tile( tile ):
    print( tile[2] )
    print( tile[0] )
    return Image.fromarray( 255 - tile[1] ).convert('L')

In [ ]:
i = 0

In [ ]:
img = show_tile( tiles[i] )
print(i)

i += 1
if i >= len(tiles):
    i = 0

img